In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset
import numpy as np
from src.evaluate_crows import evaluate_crows
from src.stereosetEvaluator import StereosetEvaluator
import os
import pickle

/Users/alessandro/Documents/Universita/Progetto_NLP/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models_names = ['distilbert/distilbert-base-uncased','albert/albert-base-v2','FacebookAI/xlm-roberta-base']
#models_names = ['albert/albert-base-v2','FacebookAI/xlm-roberta-base']

# Crows evaluation

In [3]:

for model_name in models_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    print('Computing CROWS for: ',model_name,)
    results = evaluate_crows(model,tokenizer)
    print(f' {results}')
    result_path = 'results/'+model_name.replace('/','-')
    os.makedirs(result_path, exist_ok=True)
    with open(os.path.join(result_path,'crows.pkl'),'wb') as f:
        pickle.dump(results, f)
    print('-'*25)


Computing CROWS for:  distilbert/distilbert-base-uncased


100%|██████████| 1508/1508 [10:18<00:00,  2.44it/s]


 {'general': 0.5229235880398672, 'race-color': 0.4970873786407767, 'socioeconomic': 0.5789473684210527, 'gender': 0.46564885496183206, 'disability': 0.5254237288135594, 'nationality': 0.5345911949685535, 'sexual-orientation': 0.6547619047619048, 'physical-appearance': 0.3492063492063492, 'religion': 0.6571428571428571, 'age': 0.5517241379310345}
-------------------------


Some weights of the model checkpoint at albert/albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Computing CROWS for:  albert/albert-base-v2


...

 {'general': 0.5009966777408638, 'race-color': 0.45048543689320386, 'socioeconomic': 0.6257309941520468, 'gender': 0.366412213740458, 'disability': 0.6271186440677966, 'nationality': 0.4716981132075472, 'sexual-orientation': 0.7619047619047619, 'physical-appearance': 0.49206349206349204, 'religion': 0.5333333333333333, 'age': 0.6436781609195402}
-------------------------


Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Computing CROWS for:  FacebookAI/xlm-roberta-base


100%|██████████| 1508/1508 [28:08<00:00,  1.12s/it]

 {'general': 0.4920212765957447, 'race-color': 0.4563106796116505, 'socioeconomic': 0.5847953216374269, 'gender': 0.5134099616858238, 'disability': 0.6101694915254238, 'nationality': 0.49056603773584906, 'sexual-orientation': 0.5714285714285714, 'physical-appearance': 0.5396825396825397, 'religion': 0.4095238095238095, 'age': 0.367816091954023}
-------------------------


# StereoSet IntraSentence

In [5]:
for model_name in models_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    print('Computing StereoSet (Intrasentence) for: ',model_name,)
    stereoset_evaluator = StereosetEvaluator(model, 
                                            tokenizer,
                                            './data/stereoset/test.json',
                                            device = 'mps')
    results = stereoset_evaluator.evaluate_intrasentence()
    print(f' {results}')
    result_path = 'results/'+model_name.replace('/','-')
    os.makedirs(result_path, exist_ok=True)
    with open(os.path.join(result_path,'stereoset-intra.pkl'),'wb') as f:
        pickle.dump(results, f)
    print('-'*25)

Computing StereoSet (Intrasentence) for:  distilbert/distilbert-base-uncased


100%|██████████| 6392/6392 [03:20<00:00, 31.82it/s]


 {'general': 0.6081990298857769, 'race': 0.5784873949579832, 'gender': 0.6459143968871596, 'profession': 0.6321934945788157, 'religion': 0.6153846153846154}
-------------------------


Some weights of the model checkpoint at albert/albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Computing StereoSet (Intrasentence) for:  albert/albert-base-v2


 36%|███▋      | 2333/6392 [01:44<03:01, 22.31it/s]


KeyboardInterrupt: 

# StereoSet InterSentence

In [ ]:
for model_name in models_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    print('Computing StereoSet (Intersentence) for: ',model_name,)
    stereoset_evaluator = StereosetEvaluator(model, 
                                            tokenizer,
                                            './data/stereoset/test.json',
                                            device = 'mps')
    results = stereoset_evaluator.evaluate_intersentence()
    result_path = 'results/'+model_name.replace('/','-')
    os.makedirs(result_path, exist_ok=True)
    with open(os.path.join(result_path,'stereoset-inter.pkl'),'wb') as f:
        pickle.dump(results, f)
    print('-'*25)